In [0]:
# !git config --global user.email "me@example.com"
# !git config --global user.name "nervuzz"
# !git status
# !git pull
# !git add .
# !git commit -m 'Day 5'
# !git status
# !git push

In [1]:
cd "drive/My Drive/Colab Notebooks/data_workshop_car/"

/content/drive/My Drive/Colab Notebooks/data_workshop_car


In [0]:
# !pip install --upgrade tables
# !pip install eli5
# !pip install xgboost
# !pip install hyperopt

In [5]:
import numpy as np
import pandas as pd
import xgboost as xgb
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score
from hyperopt import hp, fmin, tpe, STATUS_OK


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


# Data loading section

In [6]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

# Feature engineering

In [0]:
CAT_SUFFIX = '__cat'

for feat in df.columns:
  if not isinstance(df[feat][0] , list):
    factorized_values = df[feat].factorize()[0]
    if CAT_SUFFIX in feat:
      df[feat] = factorized_values
    else:
      df[ feat + CAT_SUFFIX ] = factorized_values

In [8]:
cat_feats = [x for x in df.columns if CAT_SUFFIX in x and 'price' not in str(x).lower()]
len(cat_feats)

151

# run_model() function

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values
  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

# XGBoost

In [0]:
xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0,
}

model = xgb.XGBRegressor(**xgb_params, silent=True)

# Improved feats & run

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.replace(' ', '').split('KM')[0]))
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.replace(' ', '').split('cm')[0]))

In [0]:
feats = ['param_rok-produkcji','param_moc','param_pojemność-skokowa','param_napęd__cat','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','seller_name__cat','feature_wspomaganie-kierownicy__cat','param_model-pojazdu__cat','param_wersja__cat','param_kod-silnika__cat','feature_system-start-stop__cat','feature_asystent-pasa-ruchu__cat','feature_czujniki-parkowania-przednie__cat','feature_łopatki-zmiany-biegów__cat','feature_regulowane-zawieszenie__cat']

In [13]:
run_model(model, feats)

(-9620.550798000319, 100.27045195136738)

# Auto-tune opts with Hyperopt

In [0]:
def obj_func(params):
  print(f"Train w params: {params}")
  
  mean_mae, score_std = run_model(xgb.XGBRegressor(**params, silent=True), feats)
  
  return {'loss': np.abs(mean_mae), 'status': STATUS_OK}

In [19]:
# space
xgb_reg_params = {
    'learning_rate': hp.choice('learning_rate', np.arange(0.05, 0.31, 0.05)),
    'max_depth': hp.choice('max_depth', np.arange(5, 16, 1, dtype=int)),
    'subsample': hp.quniform('subsample', 0.05, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.05, 1, 0.05),
    'objective': 'reg:squarederror',
    'n_estimators': 100,
    'seed': 0,
}

# run
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)

best

Train w params: {'colsample_bytree': 0.25, 'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.35000000000000003}
Train w params: {'colsample_bytree': 0.8, 'learning_rate': 0.25, 'max_depth': 13, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9}
Train w params: {'colsample_bytree': 0.9500000000000001, 'learning_rate': 0.1, 'max_depth': 11, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.35000000000000003}
Train w params: {'colsample_bytree': 0.25, 'learning_rate': 0.05, 'max_depth': 14, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.65}
Train w params: {'colsample_bytree': 0.4, 'learning_rate': 0.3, 'max_depth': 14, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.5}
Train w params: {'colsample_bytree': 0.45, 'learning_rate': 0.1, 'max_depth': 14, 'n_estimators': 100, 'objective': 'reg:squared

{'colsample_bytree': 0.55,
 'learning_rate': 1,
 'max_depth': 5,
 'subsample': 0.9500000000000001}

In [21]:

# !git pull
# !git add .
# !git commit -m 'Day 5'
# !git status
# !git push

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   day4.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	day5.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
